In [1]:
import numpy as np
import sys
import os
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
os.chdir("/home/luqiaolin/projects/Benchmarking_paper_code/cosmx_spot_data/pretrain/lung/All+SMI+Flat+data")

In [2]:
!ls
!pwd

1.csv			 Lung12  Lung5_Rep1  Lung5_Rep3  Lung9_Rep1  vis   vis2
cosMX_showcase_meta.csv  Lung13  Lung5_Rep2  Lung6	 Lung9_Rep2  vis1  vis3
/home/luqiaolin/projects/Benchmarking_paper_code/cosmx_spot_data/pretrain/lung/All+SMI+Flat+data


In [3]:
gene_expression = pd.read_csv('./Lung12/Lung12-Flat_files_and_images/Lung12_exprMat_file.csv')
cell_boundary = pd.read_csv('./Lung12/Lung12-Flat_files_and_images/Lung12_metadata_file.csv')
fov_position = pd.read_csv('./Lung12/Lung12-Flat_files_and_images/Lung12_fov_positions_file.csv')


In [4]:
x_coordinate = [x / 10000 for x in list(fov_position["x_global_px"])]
y_coordinate = [y / 10000 for y in list(fov_position["y_global_px"])]


In [5]:
def get_spot_fov_cellId_mapping(data_result, cell_boundary_fov_11):
    new_col_val = cell_boundary_fov_11.shape[0]* [0]
    cell_boundary_fov_11.insert(loc=0, column='spot_id', value=new_col_val)

    for i in range(cell_boundary_fov_11.shape[0]):
        one_row_sample = cell_boundary_fov_11.iloc[i]

        center_x = one_row_sample["CenterX_local_px"]
        center_y = one_row_sample["CenterY_local_px"]

        # Calculate spot_id based on CenterX_local_px and CenterY_local_px values
        spot_id = 1 + 4 * int((center_x - 1e-6) / 1094.4) + int((center_y - 1e-6) / 912)

        # Use spot_id as needed in your code
        # print(spot_id)
        data_result = data_result.append({'spot_id' : spot_id, 'fov' : one_row_sample["fov"], 'cell_ID' : one_row_sample["cell_ID"]}, ignore_index = True)
    return data_result
    

In [6]:
data_final_result = pd.DataFrame(columns = ['spot_id', 'fov', 'cell_ID'])
fov_ids_lst = cell_boundary['fov'].unique()
print("fov_ids_lst:", fov_ids_lst)

for fov_id in tqdm(fov_ids_lst):
    # print("fov_id:", fov_id)
    cell_boundary_fov =  cell_boundary[(cell_boundary['fov']==fov_ids_lst[fov_id-1])]
    # print("cell_boundary_fov_without_spot_id:", cell_boundary_fov.shape)
    data_final_result = get_spot_fov_cellId_mapping(data_final_result, cell_boundary_fov)


fov_ids_lst: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28]


  0%|          | 0/28 [00:00<?, ?it/s]

100%|██████████| 28/28 [01:04<00:00,  2.29s/it]


In [7]:
spot_celld_mapping = data_final_result


In [8]:
names = ['fov'] + ['spot-id=' + str(i) for i in range(1,21)]
fov_dic = {}
for i in names:
    fov_dic[i] = 0
    
fov_ids_lst = cell_boundary['fov'].unique()
spot_id_lst = [i for i in range(1,21)]


names = ['fov'] + ['spot-id=' + str(i) for i in range(1,21)]
fov_spot_cells_stats = pd.DataFrame(columns = names)


for fov_id in fov_ids_lst:
    fov_data = data_final_result[(data_final_result['fov']==fov_id)]
    
    fov_dic_sample = fov_dic
    fov_dic_sample["fov"] = fov_id
    
    for i in spot_id_lst:
        spot_id_data = fov_data[(fov_data['spot_id']==i)]
        spot_id_num = "spot-id=" + str(i)
        fov_dic_sample[spot_id_num] = spot_id_data.shape[0]
    fov_spot_cells_stats = fov_spot_cells_stats.append(fov_dic_sample, ignore_index = True)



In [9]:
def get_spot_gene_expression(spot_cell_mapping, fov_expression, spot_id):
    genes_lst = list(fov_expression.columns)[2:]
    
    cell_id_lst = spot_cell_mapping[(spot_cell_mapping['spot_id']==spot_id)]["cell_ID"].tolist()
    
    cell_gene_expression_total = len(genes_lst)*[0]
    for cell_id in cell_id_lst:
        cell_gene_expression = fov_expression[(fov_expression['cell_ID'] == cell_id)]
        cell_gene_expression = cell_gene_expression.values.tolist()[0][2:]
        cell_gene_expression_total = np.sum([cell_gene_expression_total, cell_gene_expression], axis=0).tolist()
    
    return cell_gene_expression_total
    

In [10]:

fov_ids_lst = cell_boundary['fov'].unique()
spot_id_lst = [i for i in range(1,21)]
spot_gene_expression = ["fov", "spot_id"]
genes_name_lst = list(gene_expression.columns)[2:]
spot_gene_expression = spot_gene_expression + genes_name_lst
spot_gene_expression = pd.DataFrame(columns = spot_gene_expression)

for fov_id in tqdm(fov_ids_lst):
    fov_data = data_final_result[(data_final_result['fov']==fov_id)]
    fov_gene_expression = gene_expression[(gene_expression['fov'] == fov_id)]
    
    for spot_id in spot_id_lst:
        to_append = [fov_id, spot_id]
        spot_gene_express = get_spot_gene_expression(fov_data, fov_gene_expression, spot_id)
        to_append = to_append + spot_gene_express
        a_series = pd.Series(to_append, index = spot_gene_expression.columns)
        spot_gene_expression = spot_gene_expression.append(a_series, ignore_index=True)    
  

  0%|          | 0/28 [00:00<?, ?it/s]

100%|██████████| 28/28 [01:39<00:00,  3.57s/it]


In [11]:
CosMx_cell_type = pd.read_csv('./cosMX_showcase_meta.csv')
sample_cell_count_dic = {}
sample_fov_count_dic = {}
for i in range(1, 9):
    sample_cell_count_dic[i] = 0
    sample_fov_count_dic[i] = 0
# print("cell:", sample_cell_count_dic)
# print("fov:", sample_fov_count_dic)

sample_7_index_lst = []

cell_id_lst = CosMx_cell_type["cell_ID"].tolist()
# print("len of cell_id_lst:", len(cell_id_lst))

fov_lst = []

index = 0
for i in range(len(cell_id_lst)):
    sample_id = int(cell_id_lst[i].split("_")[1])
    fov_id = int(cell_id_lst[i].split("_")[2])
    cell_id = int(cell_id_lst[i].split("_")[3])

    if sample_id == 7:
        sample_7_index_lst.append(index)
        
    sample_cell_count_dic[sample_id] =  sample_cell_count_dic[sample_id] + 1
    if sample_id == 7:
        if fov_id not in fov_lst:
            fov_lst.append(fov_id)
    index += 1

print(sample_cell_count_dic)
print(fov_lst)
print(len(sample_7_index_lst))


{1: 98002, 2: 105800, 3: 97809, 4: 89975, 5: 87606, 6: 139504, 7: 71304, 8: 81236}
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
71304


In [12]:
CosMx_cell_type_sample_7 = CosMx_cell_type.iloc[sample_7_index_lst]
CosMx_cell_type_sample_7.reset_index(drop=True, inplace=True)

for i in CosMx_cell_type_sample_7.index:
    CosMx_cell_type_sample_7.at[i,'cell_ID']= int(CosMx_cell_type_sample_7.iloc[i]["cell_ID"].split("_")[3])



In [13]:
CosMx_cell_type_sample_7

,Unnamed: 0,cell_ID,fov,Area,AspectRatio,Width,Height,Mean.CD298,Max.CD298,Mean.G,...,Panel,Diversity,totalcounts,log10totalcounts,background,remove_flagged_cells,patient,cell_type,niche,prop_tumor_in_100_neighbors
0,618697,1,1,3411,1.98,93,47,7127,16033,3000,...,980p,80,108,2.033424,0.023273,False,Lung12,neutrophil,tumor interior,0.77
1,618698,2,1,2993,1.77,78,44,13547,25859,12494,...,980p,49,70,1.845098,0.015084,False,Lung12,tumor 12,tumor interior,0.74
2,618699,3,1,5590,1.75,112,64,10353,22954,7951,...,980p,74,93,1.968483,0.020040,False,Lung12,tumor 12,tumor interior,0.72
3,618700,4,1,9932,1.19,124,104,14990,45063,9127,...,980p,209,395,2.596597,0.085117,False,Lung12,tumor 12,tumor interior,0.62
4,618701,5,1,3158,2.05,90,44,6072,16689,8497,...,980p,42,52,1.716003,0.011205,False,Lung12,tumor 12,tumor interior,0.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71299,689996,3143,28,1352,1.49,55,37,1914,10484,176,...,980p,62,88,1.944483,0.018963,False,Lung12,T CD8 naive,stroma,0.04
71300,689997,3144,28,1917,1.18,52,44,1879,5069,141,...,980p,17,23,1.361728,0.004956,False,Lung12,macrophage,stroma,0.03
71301,689998,3145,28,1999,1.27,56,44,2077,6543,276,...,980p,68,97,1.986772,0.020902,False,Lung12,fibroblast,stroma,0.09
71302,689999,3146,28,1883,1.33,56,42,1796,9458,335,...,980p,123,229,2.359835,0.049347,False,Lung12,fibroblast,stroma,0.11


In [14]:
sample_7_dic = {}
for key in CosMx_cell_type_sample_7["cell_type"].tolist():
    if key not in sample_7_dic:
        sample_7_dic[key] = 1
    else:
        sample_7_dic[key] = sample_7_dic[key] + 1
# sample_7_dic


In [15]:
cell_type_lst = list(CosMx_cell_type_sample_7['cell_type'].unique())
cell_id_lst = CosMx_cell_type_sample_7["cell_ID"].tolist()


In [16]:
def get_spot_cell_type_dic(one_spot_cell_lst, CosMx_cell_type, cell_type_dic):
    existig_cell_ids_with_celltype = CosMx_cell_type['cell_ID'].unique()
    
    for cell_id in one_spot_cell_lst:
        if cell_id in existig_cell_ids_with_celltype:
            one_cell_sample = CosMx_cell_type[(CosMx_cell_type['cell_ID']==cell_id)]
            cell_type = one_cell_sample["cell_type"].values[0]
            cell_type_dic[cell_type] = cell_type_dic[cell_type] + 1
        else:
            print("cell id not found in groud truth!!!")
        
    return cell_type_dic


In [17]:
fov_ids_lst = spot_celld_mapping['fov'].unique()
spot_id_lst = [i for i in range(1,21)]

column_name_lst = ['fov', 'spot_id'] + cell_type_lst
ground_truth_table = pd.DataFrame(columns = column_name_lst)

for fov_id in fov_ids_lst:
    fov_data = spot_celld_mapping[(spot_celld_mapping['fov']==fov_id)]

    for spot_id in spot_id_lst:
        sample_dic = {}
        for i in column_name_lst:
            sample_dic[i] = 0
        
        spot_id_data = fov_data[(fov_data['spot_id']==spot_id)]
        one_spot_cell_lst = list(spot_id_data['cell_ID'].unique()) # all cell ids for one specific spot
        
        CosMx_cell_type_sample_fov = CosMx_cell_type_sample_7[(CosMx_cell_type_sample_7['fov']==fov_id)]
        sample_dic = get_spot_cell_type_dic(one_spot_cell_lst, CosMx_cell_type_sample_fov, sample_dic)
        
        sample_dic["fov"] = fov_id
        sample_dic["spot_id"] = spot_id
        ground_truth_table = ground_truth_table.append(sample_dic, ignore_index = True)




cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud t

In [18]:
ground_truth_table

,fov,spot_id,neutrophil,tumor 12,macrophage,epithelial,endothelial,fibroblast,B-cell,pDC,...,plasmablast,NK,T CD8 naive,monocyte,Treg,tumor 9,tumor 6,T CD4 memory,tumor 13,T CD8 memory
0,1.0,1.0,3.0,5.0,0.0,11.0,3.0,0.0,2.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,2.0,2.0,4.0,2.0,11.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,3.0,6.0,3.0,1.0,11.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,4.0,10.0,4.0,0.0,5.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,5.0,9.0,1.0,0.0,7.0,3.0,0.0,2.0,0.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595,28.0,196.0,6.0,2.0,2.0,1.0,0.0,4.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5596,28.0,197.0,0.0,3.0,1.0,0.0,1.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5597,28.0,198.0,2.0,1.0,1.0,0.0,1.0,8.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5598,28.0,199.0,2.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
ground_truth_table.rename(columns = {'tumor 12':'tumor'}, inplace = True)
del ground_truth_table['tumor 9']
del ground_truth_table['tumor 13']
del ground_truth_table['tumor 6']
del ground_truth_table['tumor 5']

In [20]:
def get_spot_x_y_range(x_global_px, y_global_px, fov_id, fov_spot_coordinates):
    fov_length = 5472
    fov_width = 3648
    x_l = x_global_px
    y_l = y_global_px
    x_h = x_l + fov_length
    y_h = y_l + fov_width
    
    for i in range(1, 6):  # 5
        for j in range(1, 5):  # 4
            spot_id = (i - 1) * 4 + j
            x = x_l + (5472 / 10.0) * 1 + (5472 / 5.0) * (i - 1)
            y = y_h - (3648 / 8.0) * 1 - (3648 / 4.0) * (j - 1)
            fov_spot_coordinates = fov_spot_coordinates.append(
                {'fov': fov_id, 'spot_id': spot_id, 'x': x, 'y': y},
                ignore_index=True
            )

    fov_spot_coordinates['x'] = fov_spot_coordinates['x']
    fov_spot_coordinates['y'] = fov_spot_coordinates['y']

    # fov_spot_coordinates['x'] = fov_spot_coordinates['x'] * 0.18 * 1e-4
    # fov_spot_coordinates['y'] = fov_spot_coordinates['y'] * 0.18 * 1e-4

    return fov_spot_coordinates



In [21]:
fov_spot_coordinates = pd.DataFrame(columns = ['fov', 'spot_id', 'x', 'y'])
for fov_id in fov_lst:
    x_px = fov_position[fov_position['fov']==fov_id]["x_global_px"].values[0]
    y_px = fov_position[fov_position['fov']==fov_id]["y_global_px"].values[0]
    fov_spot_coordinates = get_spot_x_y_range(x_px, y_px, fov_id, fov_spot_coordinates)
fov_spot_coordinates

,fov,spot_id,x,y
0,1.0,1.0,-4840.977778,-156767.733333
1,1.0,2.0,-4840.977778,-157132.533333
2,1.0,3.0,-4840.977778,-157497.333333
3,1.0,4.0,-4840.977778,-157862.133333
4,1.0,5.0,-4840.977778,-158226.933333
...,...,...,...,...
5595,28.0,196.0,16774.088889,-136691.733333
5596,28.0,197.0,16774.088889,-137056.533333
5597,28.0,198.0,16774.088889,-137421.333333
5598,28.0,199.0,16774.088889,-137786.133333


In [22]:
spot_gene_expression.shape

(5600, 982)

In [23]:
spot_gene_expression = spot_gene_expression.drop(columns = ["fov", "spot_id"])
ground_truth_table = ground_truth_table.drop(columns = ["fov", "spot_id"])
fov_spot_coordinates = fov_spot_coordinates.drop(columns = ["fov", "spot_id"])



import anndata as ad
st_adata = ad.AnnData(X = spot_gene_expression.values, obs = ground_truth_table, var = pd.DataFrame(index = list(spot_gene_expression.columns)), dtype=int)
st_adata.obsm["spatial"] = fov_spot_coordinates.values

spot_sums = np.sum(st_adata.X, axis=1)
mask = spot_sums > 100
filtered_data = st_adata[mask]

file_path = "/home/luqiaolin/projects/Benchmarking_paper_code/pseudo_spot_generation/cosmx_lung/Lung12.h5ad"
filtered_data.write_h5ad(file_path)



